FEATURE ENGINEERING

In [36]:
import numpy as np
import csv
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
node_info = pd.read_csv("../data/node_information.csv", header=None)
test_set = pd.read_csv("../data/test.txt", sep=" ", header=None, names=['source', 'target'])
train_set = pd.read_csv("../data/train.txt", sep=" ", header=None, names=['source', 'target', 'label'])

# Graph creation
G = nx.from_pandas_edgelist(train_set, 'source', 'target', 'label')

In [38]:
node_info

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592,7584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3593,7589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,7593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3595,7594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
def salton_similarity(G, edges):
    for u, v in edges:
        common_neighbors = len(list(nx.common_neighbors(G, u, v)))
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        yield u, v, common_neighbors / ((degree_u * degree_v) ** 0.5)

def sorenson_similarity(G, edges):
    for u, v in edges:
        common_neighbors = len(list(nx.common_neighbors(G, u, v)))
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        yield u, v, 2 * common_neighbors / (degree_u + degree_v)


def hub_promoted_similarity(G, edges):
    for u, v in edges:
        common_neighbors = len(list(nx.common_neighbors(G, u, v)))
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        yield u, v, common_neighbors / min(degree_u, degree_v)

def hub_depressed_similarity(G, edges):
    for u, v in edges:
        common_neighbors = len(list(nx.common_neighbors(G, u, v)))
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        yield u, v, common_neighbors / max(degree_u, degree_v)

def adamic_adar_index(G, edges):
    for u, v in edges:
        score = 0
        for w in nx.common_neighbors(G, u, v):
            degree_w = G.degree(w)
            if degree_w > 1:
                score += 1 / np.log(degree_w)
        yield u, v, score


# Creer une fonction qui ajoute des features (degree, centrality, clustering) à un graphes et qui merge node_info avec le graphe
def add_node_attributes(df, node_info):
     # Créer un graphe à partir du df
     if 'label' in df.columns:
        G = nx.from_pandas_edgelist(df, 'source', 'target', 'label')
     else:
        G = nx.from_pandas_edgelist(df, 'source', 'target')

     # Calculer les caractéristiques
     df['degree_source'] = df['source'].apply(lambda x: G.degree(x))
     df['centrality_source'] = df['source'].apply(lambda x: nx.degree_centrality(G)[x])
     #df['clustering_source'] = df['source'].apply(lambda x: nx.clustering(G)[x])

     df['degree_target'] = df['target'].apply(lambda x: G.degree(x))
     df['centrality_target'] = df['target'].apply(lambda x: nx.degree_centrality(G)[x])
     #df['clustering_target'] = df['target'].apply(lambda x: nx.clustering(G)[x])

     # Détecter les communautés et créer une caractéristique de communauté
     communities = nx_community.greedy_modularity_communities(G)
     community_map = {}
     for i, community in enumerate(communities):
          for node in community:
               community_map[node] = i
     df['community_source'] = df['source'].apply(lambda x: community_map[x])
     df['community_target'] = df['target'].apply(lambda x: community_map[x])

     # Calculer le coefficient de Jaccard
     df['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df[['source', 'target']].values)]

     # Calculer le coefficient de similarité de Salton
     df['salton'] = [i[2] for i in salton_similarity(G, df[['source', 'target']].values.tolist())]

     # Calculer le coefficient de similarité de Sorenson
     df['sorenson'] = [i[2] for i in sorenson_similarity(G, df[['source', 'target']].values.tolist())]

     # Calculer le coefficient de similarité de Hub Promoted
     df['hub_promoted'] = [i[2] for i in hub_promoted_similarity(G, df[['source', 'target']].values.tolist())]

     # Calculer le coefficient de similarité de Hub Depressed
     df['hub_depressed'] = [i[2] for i in hub_depressed_similarity(G, df[['source', 'target']].values.tolist())]

     # Calculer le coefficient de similarité de Leicht-Holme-Newman
     df['leicht_holme_newman'] = [i[2] for i in nx.preferential_attachment(G, df[['source', 'target']].values)]

     # Calculer le coefficient de similarité de adamic_adar
     df['adamic_adar'] = [i[2] for i in adamic_adar_index(G, df[['source', 'target']].values.tolist())]

     # Calculer le coefficient de similarité de resource_allocation_index
     df['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, df[['source', 'target']].values)]

     # Calculer les common neighbors
     df['common_neighbors'] = df.apply(lambda x: len(list(nx.common_neighbors(G, x['source'], x['target']))), axis=1)

     df['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, df[['source', 'target']].values)]

     # Calculer les common neighbors
     df['common_neighbors'] = df.apply(lambda x: len(list(nx.common_neighbors(G, x['source'], x['target']))), axis=1)

    # Calculate eigenvector centrality
     eigenvector_centrality = nx.eigenvector_centrality_numpy(G)
     df['eigenvector_centrality_source'] = df['source'].apply(lambda x: eigenvector_centrality.get(x, 0))
     df['eigenvector_centrality_target'] = df['target'].apply(lambda x: eigenvector_centrality.get(x, 0))

     # Calculate cosine similarity
     def calculate_cosine_similarity(node_encodings, source_node, target_node):
        source_vector = node_encodings.loc[source_node].values.reshape(1, -1)
        target_vector = node_encodings.loc[target_node].values.reshape(1, -1)
        return cosine_similarity(source_vector, target_vector)[0][0]
     
     node_info= node_info.rename(columns={0: 'node_id'})
     
     node_info.set_index('node_id', inplace=True)
     df['cosine_similarity'] = df.apply(lambda row: calculate_cosine_similarity(node_info, row['source'], row['target']), axis=1)

     # Fusionner node_info avec le df
     df = df.merge(node_info, left_on='source', right_on='node_id', how='left')
     # rename toutes les colonnes de node_info ajouter par "nom_col"+"source"
     df.rename(columns={col: str(col) + '_source' for col in node_info.columns[1:]}, inplace=True)


     df = df.merge(node_info, left_on='target', right_on='node_id', how='left')
     # rename toutes les colonnes de node_info ajouter par "nom_col"+"source"
     df.rename(columns={col: str(col) + '_target' for col in node_info.columns[1:]}, inplace=True)

     # Reset index 0 to n-1
     df.reset_index(drop=True, inplace=True)
     return df

# Ajouter les caractéristiques au train_set
train_set = add_node_attributes(train_set, node_info)
test_set = add_node_attributes(test_set, node_info)

train_set.head()


,source,target,label,degree_source,centrality_source,degree_target,centrality_target,community_source,community_target,jaccard,...,923_target,924_target,925_target,926_target,927_target,928_target,929_target,930_target,931_target,932_target
0,939,3809,1,4,0.001112,361,0.100389,2,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2442,5784,1,11,0.003059,4,0.001112,3,3,0.071429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,179,3809,1,8,0.002225,361,0.100389,0,0,0.005450,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,857,2280,1,9,0.002503,21,0.005840,0,4,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1358,5722,1,3,0.000834,17,0.004727,2,2,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
train_set.to_csv('../data/train_set_final.csv', index=False)
test_set.to_csv('../data/test_set_final.csv', index=False)